In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
#import Segmentation
import time
import preds

importing Jupyter notebook from preds.ipynb
importing Jupyter notebook from Captcha_seg.ipynb
1080


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 17s 20ms/step - loss: 3.5885 - acc: 0.0243 - val_loss: 3.5702 - val_acc: 0.0972
Epoch 2/20
864/864 [==============================] - 6s 7ms/step - loss: 3.4287 - acc: 0.0741 - val_loss: 3.1722 - val_acc: 0.1343
Epoch 3/20
864/864 [==============================] - 6s 7ms/step - loss: 2.7239 - acc: 0.2280 - val_loss: 2.6701 - val_acc: 0.2500
Epoch 4/20
864/864 [==============================] - 6s 7ms/step - loss: 1.9986 - acc: 0.4074 - val_loss: 2.2769 - val_acc: 0.3472
Epoch 5/20
864/864 [==============================] - 6s 7ms/step - loss: 1.4641 - acc: 0.5567 - val_loss: 1.8317 - val_acc: 0.4861
Epoch 6/20
864/864 [==============================] - 6s 7ms/step - loss: 0.9978 - acc: 0.7025 - val_loss: 1.8146 - val_acc: 0.5370
Epoch 7/20
864/864 [==============================] - 7s 9ms/step - loss: 0.6854 - acc: 0.7870 - val_loss: 1.9422 - val_acc: 0.5278
Epoch 8/20
864/864 [========

In [2]:
CATEGORIES = set()
names =[]
for i in os.listdir('test/'):
    names.append(i.split('.')[0])
for i in names:
    for x in list(i):
        CATEGORIES.add(x)
CATEGORIES=list(CATEGORIES)
print(CATEGORIES)

['c', 'j', 'd', '4', 'e', 'y', '2', 'x', 'r', 'g', 'm', '6', '3', '9', 'u', 'h', 'p', 'a', 'v', 's', 'n', '7', 'f', 't', 'w', 'z', 'k', '8', 'l', 'b', '5']


In [3]:
#print(len(train_data))

In [4]:
import random

#random.shuffle(train_data)

In [5]:
#for sample in train_data[:10]:
#    print(sample[1])

In [6]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


In [7]:
#loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

In [8]:
#print(loss_value, metrics_value)

In [9]:
import random

In [10]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
done = []
list_ = preds.predictions('test',CATEGORIES)
CAPTCHA_DIR = list_[0]
for x in CAPTCHA_DIR:
    captchas = random.choice(CAPTCHA_DIR)
    if captchas in done:
        continue
    else:
        done.append(captchas)
        start_time = time.time()
        predict_list= []
        print('\n')
        print('Actual: ',captchas[0][0])

        files = []
        for segments in list_:
            files.append(segments)

        #for segments in files:
        #    filepath = (SEGMENTS_DIR,segments)
        #    prediction = model.predict([prepare(filepath)])
        #    predict_list.append(CATEGORIES[np.argmax(prediction[0])])

        elapsed_time = time.time()-start_time
        time.sleep(captchas[1])
        total_time += elapsed_time
        final_prediction = ''.join(predict_list)
        #if captcha_name[0]== final_prediction:
        #    score +=1
        score = list_[1]
        print('Predicted:',captchas[0][1],'\n')
        print('Elapsed time= ',captchas[1],'s')
        total_time +=captchas[1]
        print('<------------------------------->')    
    

print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',len(list_[0]))
print('Total time taken= ',total_time,'s')
print('Accuracy= ', (score/len(list_[0])*100))
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  gmreg
Predicted: gmreg 

Elapsed time=  0.44803189099548335 s
<------------------------------->


Actual:  4l75m
Predicted: 68sj 

Elapsed time=  0.11879154626900057 s
<------------------------------->


Actual:  rpfk2
Predicted: rpfk2 

Elapsed time=  0.06241245804168549 s
<------------------------------->


Actual:  uzdsl
Predicted: mrhy 

Elapsed time=  0.9847124356274999 s
<------------------------------->


Actual:  2my3y
Predicted: 2my3y 

Elapsed time=  0.6794581245947848 s
<------------------------------->


Actual:  skxkn
Predicted: s5l2 

Elapsed time=  0.8520804830452834 s
<------------------------------->


Actual:  rw9dx
Predicted: rw9dx 

Elapsed time=  0.24028758745199863 s
<------------------------------->


Actual:  vsx3g
Predicted: vsx3g 

Elapsed time=  0.866361199542424 s
<------------------------------->


Actual:  xdn95
Predicted: rkvc 

Elapsed time=  0.1286142892212453 s
<------------------------------->


Actual:  aep95
Predicted: aep95 

Elapsed tim

Predicted: n7k4r 

Elapsed time=  0.9058840310910509 s
<------------------------------->


Actual:  4nub3
Predicted: upmj 

Elapsed time=  0.2884564315067373 s
<------------------------------->


Actual:  srmtc
Predicted: srmtc 

Elapsed time=  0.03656410526237841 s
<------------------------------->


Actual:  dseum
Predicted: dseum 

Elapsed time=  0.20717673711480755 s
<------------------------------->


Actual:  6a23d
Predicted: 6a23d 

Elapsed time=  0.14875379537927835 s
<------------------------------->


Actual:  k888e
Predicted: k888e 

Elapsed time=  0.6258653373463904 s
<------------------------------->


Actual:  at2ah
Predicted: uwwc 

Elapsed time=  0.6512880039060448 s
<------------------------------->


Actual:  p4c5g
Predicted: p4c5g 

Elapsed time=  0.8965987621259742 s
<------------------------------->


Actual:  c9e2z
Predicted: zvu9 

Elapsed time=  0.19802772726579865 s
<------------------------------->


Actual:  6xpmk
Predicted: mjf5 

Elapsed time=  0.2276041981

Predicted: dxsue 

Elapsed time=  0.5346043003775196 s
<------------------------------->


Actual:  4vkn7
Predicted: 4vkn7 

Elapsed time=  0.8019768035985555 s
<------------------------------->


Actual:  5zy3b
Predicted: rtgd 

Elapsed time=  0.3156763274392089 s
<------------------------------->


Actual:  w5lbn
Predicted: sbff 

Elapsed time=  0.9878819746256253 s
<------------------------------->


Actual:  agke4
Predicted: 92av 

Elapsed time=  0.5812608040298859 s
<------------------------------->


Actual:  anm53
Predicted: fjy4 

Elapsed time=  0.15083027991164288 s
<------------------------------->


Actual:  uc925
Predicted: uc925 

Elapsed time=  0.8622799451389905 s
<------------------------------->


Actual:  uh58e
Predicted: uh58e 

Elapsed time=  0.8793665994016536 s
<------------------------------->


Actual:  wtecl
Predicted: wtecl 

Elapsed time=  0.25606555372126116 s
<------------------------------->


Actual:  csuru
Predicted: a5ha 

Elapsed time=  0.6805842634910

Predicted: tp9vx 

Elapsed time=  0.4959754588707582 s
<------------------------------->


Actual:  tg39l
Predicted: j5da 

Elapsed time=  0.4551948917573708 s
<------------------------------->


Actual:  52lhl
Predicted: 52lhl 

Elapsed time=  0.9502003880772114 s
<------------------------------->


Actual:  6g3yk
Predicted: 6g3yk 

Elapsed time=  0.3183272568820944 s
<------------------------------->


Actual:  grapp
Predicted: grapp 

Elapsed time=  0.06637440291907071 s
<------------------------------->


Actual:  x8fmj
Predicted: x8fmj 

Elapsed time=  0.18050337549221762 s
<------------------------------->
Total number of CAPTCHAS broken:  272
Total number of CAPTCHAS attempted:  398
Total time taken=  122.63040883844663 s
Accuracy=  68.34170854271356
